# 🧠 YOLOv8 Training on Colab

In [1]:
# Cài đặt YOLOv8
!pip install ultralytics==8.2.5
!pip install mlflow
!pip install torch==2.5.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [2]:
# Mount Google Drive nếu cần
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Giải nén dataset từ Google Drive
!unzip /content/drive/MyDrive/dispatch/dataset.zip -d /content/dataset
!ls /content/dataset/Detection/


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: /content/dataset/Detection/train/labels/img_001196.txt  
  inflating: /content/dataset/Detection/train/labels/img_001000.txt  
  inflating: /content/dataset/Detection/train/labels/img_000759.txt  
  inflating: /content/dataset/Detection/train/labels/img_000646.txt  
  inflating: /content/dataset/Detection/train/labels/img_000262.txt  
  inflating: /content/dataset/Detection/train/labels/img_000431.txt  
  inflating: /content/dataset/Detection/train/labels/img_001068.txt  
  inflating: /content/dataset/Detection/train/labels/img_000825.txt  
  inflating: /content/dataset/Detection/train/labels/img_000490.txt  
  inflating: /content/dataset/Detection/train/labels/img_000414.txt  
  inflating: /content/dataset/Detection/train/labels/img_001141.txt  
  inflating: /content/dataset/Detection/train/labels/img_000446.txt  
  inflating: /content/dataset/Detection/train/labels/img_000394.txt  
  inflating: /content/dataset/Det

In [4]:
# 🔕 Disable W&B logging before importing Ultralytics
!pip uninstall -y wandb
import os
os.environ["WANDB_DISABLED"] = "true"

Found existing installation: wandb 0.20.1
Uninstalling wandb-0.20.1:
  Successfully uninstalled wandb-0.20.1


In [7]:
# Huấn luyện YOLOv8 và log vào MLflow
from ultralytics import YOLO
import mlflow
import torch

mlflow.set_tracking_uri("https://adb660114e93.ngrok-free.app")
mlflow.set_experiment("dispatch-pipeline")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with mlflow.start_run(run_name="yolov8_detection_v1"):
    model = YOLO("yolov8m.pt")
    results = model.train(
        data="/content/dataset/Detection/dataset.yaml",
        epochs=100,
        imgsz=640,
        batch=16,
        patience=10,
        device=device,
        name="yolov8m_dispatch_colab"
    )
    mlflow.log_params({
        "model": "yolov8m", "epochs": 100, "imgsz": 640, "batch": 16
    })
    mlflow.log_artifacts("runs/detect/yolov8m_dispatch_colab", artifact_path="detect_artifacts")


New https://pypi.org/project/ultralytics/8.3.163 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/dataset/Detection/dataset.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=yolov8m_dispatch_colab2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

train: Scanning /content/dataset/Detection/train/labels.cache... 1040 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



/usr/local/lib/python3.11/dist-packages/ultralytics/data/augment.py:891: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning /content/dataset/Detection/val/labels.cache... 261 images, 0 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8m_dispatch_colab2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)


...

     58/100      7.28G     0.5346     0.3279      1.024         41        640: 100%|██████████| 65/65 [00:34<00:00,  1.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.36it/s]

                   all        261        374      0.993      0.981      0.991      0.903
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 48, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



58 epochs completed in 0.760 hours.
Optimizer stripped from runs/detect/yolov8m_dispatch_colab2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/yolov8m_dispatch_colab2/weights/best.pt, 52.0MB

Validating runs/detect/yolov8m_dispatch_colab2/weights/best.pt...
Ultralytics YOLOv8.2.5 🚀 Python-3.11.13 torch-2.5.1+cu118 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.41it/s]


                   all        261        374      0.988      0.989      0.994      0.907
                  dish        261        175      0.992      0.994      0.995       0.91
                  tray        261        199      0.985      0.984      0.993      0.904
Speed: 0.3ms preprocess, 9.8ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/yolov8m_dispatch_colab2
🏃 View run yolov8_detection_v1 at: https://adb660114e93.ngrok-free.app/#/experiments/772886176333578108/runs/5daa2cbc78f54afca449f62c73b22493
🧪 View experiment at: https://adb660114e93.ngrok-free.app/#/experiments/772886176333578108
MLflow: results logged to https://adb660114e93.ngrok-free.app
MLflow: disable with 'yolo settings mlflow=False'


In [8]:
!zip -r runs_yolo.zip runs/
from google.colab import files
files.download("runs_yolo.zip")

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/yolov8m_dispatch_colab2/ (stored 0%)
  adding: runs/detect/yolov8m_dispatch_colab2/labels_correlogram.jpg (deflated 43%)
  adding: runs/detect/yolov8m_dispatch_colab2/val_batch2_pred.jpg (deflated 6%)
  adding: runs/detect/yolov8m_dispatch_colab2/train_batch1.jpg (deflated 2%)
  adding: runs/detect/yolov8m_dispatch_colab2/confusion_matrix.png (deflated 37%)
  adding: runs/detect/yolov8m_dispatch_colab2/val_batch0_labels.jpg (deflated 9%)
  adding: runs/detect/yolov8m_dispatch_colab2/labels.jpg (deflated 33%)
  adding: runs/detect/yolov8m_dispatch_colab2/PR_curve.png (deflated 27%)
  adding: runs/detect/yolov8m_dispatch_colab2/P_curve.png (deflated 20%)
  adding: runs/detect/yolov8m_dispatch_colab2/train_batch2.jpg (deflated 2%)
  adding: runs/detect/yolov8m_dispatch_colab2/results.csv (deflated 85%)
  adding: runs/detect/yolov8m_dispatch_colab2/events.out.tfevents.1751989376.cb4b85b02580.589.1 (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>